* Steps to be followed Importing necessary LIbrares
* Creating S3 buckets
* mapping train and test data in S3
* mapping the path of the models in S3

In [221]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

# sagemaker: AWS SDK for training and deploying machine learning models.
# boto3: AWS SDK for Python to manage AWS services like S3, EC2, and SageMaker.
# get_image_uri: A utility function to get the container URI for built-in SageMaker algorithms.
# s3_input: A class for specifying S3 locations for training/testing data.
# Session: Manages interactions with SageMaker.

In [222]:
# bucket_name: Placeholder for your S3 bucket name. It should be unique.
# my_region: Captures the current AWS region of the instance using boto3.
# print(my_region): Displays the region to verify.
# session: Creates a Boto3 session for the specified region.
# sagemaker_client: Creates a client for interacting with SageMaker resources in the given region.

bucket_name = 'buckshhh' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)
session = boto3.Session(region_name=my_region)
sagemaker_client = session.client("sagemaker", region_name=my_region)

ap-northeast-1


[01/25/25 08:53:20] INFO     Found credentials from IAM Role:                                   ]8;id=924920;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=779513;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [223]:
import boto3

s3 = boto3.client('s3', region_name=my_region)  # Use `client` instead of `resource` for bucket creation.

try:
    if my_region == 'us-east-1':
        # No LocationConstraint for us-east-1
        s3.create_bucket(Bucket=bucket_name)
    else:
        # Add LocationConstraint for other regions
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': my_region}
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error:', e)


S3 error: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [224]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)


s3://buckshhh/xgboost-as-a-built-in-algo/output


# Downloading The Dataset And Storing in S3


In [225]:

import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
     
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)


Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.
(28831, 61) (12357, 61)


In [226]:
### Saving Train And Test Into Buckets
## We start with Train Data

from sagemaker.inputs import TrainingInput
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
     


[01/25/25 08:53:23] INFO     Found credentials from IAM Role:                                   ]8;id=858585;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=701246;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [227]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')
     


                    INFO     Found credentials from IAM Role:                                   ]8;id=709819;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=887423;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

# Building Models Xgboot- Inbuilt Algorithm


In [228]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
# get_image_uri retrieves the Docker image for XGBoost.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')


[01/25/25 08:53:25] WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=19041;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=59326;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Defaulting to only available Python version: py3                     ]8;id=156184;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=259086;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#608\608]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=196797;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=757821;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#532\532]8;;\

In [229]:

# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }


In [230]:

# boto3: AWS SDK for Python, used to interact with AWS services like SageMaker and S3.
# sagemaker.Session: Manages interactions with Amazon SageMaker, such as setting up sessions, managing training jobs, and deploying models.
# sagemaker.estimator.Estimator: High-level class to define and manage training jobs, including specifying input data, algorithms, hyperparameters, and compute resources.


import boto3
from sagemaker import Session
from sagemaker.estimator import Estimator

# Create a boto3 session with a valid region name
boto_session = boto3.Session(region_name="ap-northeast-1")

# Initialize the SageMaker session
sagemaker_session = Session(boto_session=boto_session)


# Get the SageMaker role ARN (ensure this is correctly configured in your environment)
role = sagemaker_session.get_caller_identity_arn()

# Define the estimator
estimator = Estimator(
    image_uri=container,
    hyperparameters=hyperparameters,
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    volume_size=5,  # 5 GB
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600,
    sagemaker_session=sagemaker_session  # Pass the session explicitly
)


[01/25/25 08:53:27] INFO     Found credentials from IAM Role:                                   ]8;id=320012;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=743680;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [231]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})


[01/25/25 08:53:28] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=331034;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=7185;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=631647;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=921203;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-01-25-08-53-28-039                                             

2025-01-25 08:53:29 Starting - Starting the training job...
..25-01-25 08:53:43 Starting - Preparing the instances for training.
..25-01-25 08:54:21 Downloading - Downloading the training image.
.[2025-01-25 08:55:12.699 ip-10-0-73-90.ap-northeast-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[08:55:12] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/d

# Deploy Machine Learning Model As Endpoints


In [232]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')


[01/25/25 08:55:45] INFO     Creating model with name: sagemaker-xgboost-2025-01-25-08-55-45-466    ]8;id=359559;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=801661;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[01/25/25 08:55:46] INFO     Creating endpoint-config with name                                     ]8;id=242397;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=248154;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             sagemaker-xgboost-2025-01-25-08-55-45-466                                             

                    INFO     Creating endpoint with name sagemaker-xgboost-2025-01-25-08-55-45-466  ]8;id=733943;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=925859;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!

# Prediction of the Test Data


In [233]:
 from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)


(12357,)


In [234]:
predictions_array


array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [235]:
# The model's performance is evaluated using a confusion matrix:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

# It compares the predicted results with the actual values, showing how well the model performed (e.g., accuracy, false positives, false negatives).




Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



# Deleting The Endpoints


In [242]:
print(xgb_predictor.endpoint)


[01/25/25 09:03:33] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=383194;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=978075;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

sagemaker-xgboost-2025-01-25-08-55-45-466


In [243]:
import boto3

sagemaker_client = boto3.client('sagemaker', region_name='ap-northeast-1')
response = sagemaker_client.list_endpoints()
for endpoint in response['Endpoints']:
    print(endpoint['EndpointName'])
endpoint_name = xgb_predictor.endpoint
print(endpoint_name)

[01/25/25 09:03:36] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=834985;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=848712;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

sagemaker-xgboost-2025-01-25-08-55-45-466


In [245]:
response = sagemaker_client.list_endpoints(NameContains=endpoint_name)
if any(endpoint['EndpointName'] == endpoint_name for endpoint in response['Endpoints']):
    sagemaker_session.delete_endpoint(endpoint_name)
    print(f"Deleted endpoint: {endpoint_name}")
else:
    print(f"Endpoint '{endpoint_name}' does not exist or has already been deleted.")



Endpoint 'sagemaker-xgboost-2025-01-25-08-55-45-466' does not exist or has already been deleted.


In [244]:
# response = sagemaker_client.list_endpoints(NameContains=endpoint_name)
# if any(endpoint['EndpointName'] == endpoint_name for endpoint in response['Endpoints']):
#     sagemaker_session.delete_endpoint(endpoint_name)
# else:
#     print(f"Endpoint '{endpoint_name}' does not exist or has already been deleted.")




# # This removes the live endpoint and deletes all the files from the S3 bucket to avoid unnecessary storage costs.

boto_session = boto3.Session(region_name='ap-northeast-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

# Now delete the endpoint
sagemaker_session.delete_endpoint(endpoint_name)  


[01/25/25 09:03:58] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=341619;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=602677;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Found credentials from IAM Role:                                   ]8;id=463592;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=490994;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[01/25/25 09:03:59] INFO     Deleting endpoint with name: sagemaker-xgboost-2025-01-25-08-55-45-466 ]8;id=399622;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=224917;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 sagemaker_session = sagemaker.Session(boto_session=boto_session)                            │
│   15                                                                                             │
│   16 # Now delete the endpoint                                                                   │
│ ❱ 17 sagemaker_session.delete_endpoint(endpoint_name)                                            │
│   18                                                                                             │
│   19 bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)                                 │
│   20 bucket_to_delete.objects.all().delete()                                                     │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:4856 in  │
│ delete_endpoint                                                                                  │
│                                                                                                  │
│   4853 │   │   │   endpoint_name (str): Name of the Amazon SageMaker ``Endpoint`` to delete.     │
│   4854 │   │   """                                                                               │
│   4855 │   │   logger.info("Deleting endpoint with name: %s", endpoint_name)                     │
│ ❱ 4856 │   │   self.sagemaker_client.delete_endpoint(EndpointName=endpoint_name)                 │
│   4857 │                                                                                         │
│   4858 │   def delete_endpoint_config(self, endpoint_config_name):                               │
│   4859 │   │   """Delete an Amazon SageMaker endpoint configuration.                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:569 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:1023 in    │
│ _make_api_call                                                                                   │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, opera

In [247]:
region_name = "ap-northeast-1"
bucket_to_delete = boto3.resource('s3', region_name=region_name).Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '64CAE0CZG8X12DVJ',
   'HostId': '0+fLtDBioWbeiI09Wt8EOK3UQCg4/w1YoXl5G1vC/BsFD2QEq18KWJ6ZSuIQlr6Ds5NdlKE709o=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '0+fLtDBioWbeiI09Wt8EOK3UQCg4/w1YoXl5G1vC/BsFD2QEq18KWJ6ZSuIQlr6Ds5NdlKE709o=',
    'x-amz-request-id': '64CAE0CZG8X12DVJ',
    'date': 'Sat, 25 Jan 2025 09:07:31 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-01-24-11-13-42-135/debug-output/claim.smd'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-01-25-08-53-28-039/debug-output/events/000000000040/000000000040_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-01-24-11-13-42-135/debug-output/events/000000000000/000000000000_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-a